In [1]:
import pandas as pd 
import numpy as np
import sqlite3
import pickle
pd.set_option('display.max_columns', None)
from functools import reduce 


In [2]:
conn = sqlite3.connect('database.db')

In [3]:

query = '''
    SELECT DISTINCT "Month", "LSOA name", "LSOA code", "Falls within", COUNT() AS amount_of_crimes
    FROM street2011
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code", "Falls within", COUNT() AS amount_of_crimes
    FROM street2012
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code","Falls within", COUNT() AS amount_of_crimes
    FROM street2013
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code","Falls within", COUNT() AS amount_of_crimes
    FROM street2014
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code","Falls within", COUNT() AS amount_of_crimes
    FROM street2015
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code","Falls within", COUNT() AS amount_of_crimes
    FROM street2016
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code","Falls within", COUNT() AS amount_of_crimes
    FROM street2017
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code", "Falls within", COUNT() AS amount_of_crimes
    FROM street2018
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code", "Falls within", COUNT() AS amount_of_crimes
    FROM street2019
    GROUP BY "Month", "LSOA code"
    '''
dataframe2 = pd.read_sql_query(query, conn)

In [11]:
#Train
crime_per_district = dataframe2.copy()
crime_per_district["Falls within"] = crime_per_district["Falls within"].str.rsplit(' ',1).str[0]
crime_per_district = crime_per_district.rename(columns={'LSOA name': 'name', 'LSOA code': 'Geo code', 'Falls within':'Force Name'})

In [12]:
crime_per_district['year'] = pd.to_datetime(crime_per_district['Month'], format='%Y-%m').dt.year
crime_per_district['month'] = pd.to_datetime(crime_per_district['Month'], format='%Y-%m').dt.month

In [13]:
crime_per_district['prev_month_crimes'] = crime_per_district.groupby(['Geo code'])['amount_of_crimes'].shift(1)
crime_per_district['prev_year_crimes'] = crime_per_district.groupby(['Geo code', 'month'])['amount_of_crimes'].shift(1)


In [14]:
crime_per_district = crime_per_district[crime_per_district['year']>2011]

In [8]:
crime_per_district.dropna(inplace=True)

In [5]:
del dataframe2

# Police per district (yearly)

In [42]:
df_police = pd.read_excel('Number of police officers per district per year.ods', engine='odf')
df_police = df_police.rename(columns={'As at 31 March...': 'year', 'Geo code': 'Geo code police'})

In [43]:
df_police2 = df_police[df_police['year']>2010]
df_police3 = df_police2[df_police2['year']<2020]
df_police4 = df_police3[df_police3['Worker type']=='Police Officer']
df_police_final = df_police4.groupby(['year','Force Name', 'Geo code police'])[['Total (headcount)']].sum()

In [44]:
del df_police
del df_police2
del df_police3
del df_police4

# Population LSOA (yearly)

In [45]:
df_2011 = pd.read_excel('people2011.xlsx')
df_2012 = pd.read_excel('people2012.xlsx')
df_2013 = pd.read_excel('people2013.xlsx')
df_2014 = pd.read_excel('people2014.xlsx')
df_2015 = pd.read_excel('people2015.xlsx')
df_2016 = pd.read_excel('people2016.xlsx')
df_2017 = pd.read_excel('people2017.xlsx')
df_2018 = pd.read_excel('people2018.xlsx')
df_2019 = pd.read_excel('people2019.xlsx')

C:\Users\blaze\Anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [46]:
df_2011 = df_2011[['Area Codes', 'All Ages']]
df_2012 = df_2012[['Area Codes', 'All Ages']]
df_2013 = df_2013[['Area Codes', 'All Ages']]
df_2014 = df_2014[['Area Codes', 'All Ages']]
df_2015 = df_2015[['Area Codes', 'All Ages']]
df_2016 = df_2016[['Area Codes', 'All Ages']]
df_2017 = df_2017[['Area Codes', 'All Ages']]
df_2018 = df_2018[['Area Codes', 'All Ages']]
df_2019 = df_2019[['LSOA Code', 'All Ages']]
df_2011 = df_2011.rename(columns={'All Ages':'Population 2011'})
df_2012 = df_2012.rename(columns={'All Ages':'Population 2012'})
df_2013 = df_2013.rename(columns={'All Ages':'Population 2013'})
df_2014 = df_2014.rename(columns={'All Ages':'Population 2014'})
df_2015 = df_2015.rename(columns={'All Ages':'Population 2015'})
df_2016 = df_2016.rename(columns={'All Ages':'Population 2016'})
df_2017 = df_2017.rename(columns={'All Ages':'Population 2017'})
df_2018 = df_2018.rename(columns={'All Ages':'Population 2018'})
df_2019 = df_2019.rename(columns={'LSOA Code': 'Area Codes', 'All Ages':'Population 2019'})

In [47]:
data_frames = [df_2011, df_2012, df_2013,df_2014,df_2015,df_2016,df_2017,df_2018,df_2019]

In [48]:
df_pop_merged =  reduce(lambda  left,right: pd.merge(left,right,on=['Area Codes'], how='outer'), data_frames)

In [49]:
del df_2011
del df_2012
del df_2013
del df_2014
del df_2015
del df_2016
del df_2017
del df_2018
del df_2019


In [50]:
df_pop_merged2 = df_pop_merged.copy()
df_pop_merged2 = df_pop_merged2.loc[df_pop_merged.index.repeat(8)]
df_pop_merged2['year']= 2012
df_pop_merged2['LSOA population']= 0
df_pop_merged2.reset_index(inplace=True)

In [51]:
del df_pop_merged

In [52]:
for i in range (1, df_pop_merged2.shape[0]):
    if df_pop_merged2['index'][i] == df_pop_merged2['index'][i-1]:
        df_pop_merged2['year'][i] = df_pop_merged2['year'][i-1]+1

C:\Users\blaze\AppData\Local\Temp/ipykernel_11172/4108131678.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pop_merged2['year'][i] = df_pop_merged2['year'][i-1]+1


In [53]:
for i in range (0, df_pop_merged2.shape[0]):
    if df_pop_merged2['year'][i]==2012:
        df_pop_merged2['LSOA population'][i] = df_pop_merged2['Population 2011'][i]
    if df_pop_merged2['year'][i]==2013:
        df_pop_merged2['LSOA population'][i] = df_pop_merged2['Population 2012'][i]
    if df_pop_merged2['year'][i]==2014:
        df_pop_merged2['LSOA population'][i] = df_pop_merged2['Population 2013'][i]
    if df_pop_merged2['year'][i]==2015:
        df_pop_merged2['LSOA population'][i] = df_pop_merged2['Population 2014'][i]
    if df_pop_merged2['year'][i]==2016:
        df_pop_merged2['LSOA population'][i] = df_pop_merged2['Population 2015'][i]
    if df_pop_merged2['year'][i]==2017:
        df_pop_merged2['LSOA population'][i] = df_pop_merged2['Population 2016'][i]
    if df_pop_merged2['year'][i]==2018:
        df_pop_merged2['LSOA population'][i] = df_pop_merged2['Population 2017'][i]
    if df_pop_merged2['year'][i]==2019:
        df_pop_merged2['LSOA population'][i] = df_pop_merged2['Population 2018'][i] 

C:\Users\blaze\AppData\Local\Temp/ipykernel_11172/4071623889.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pop_merged2['LSOA population'][i] = df_pop_merged2['Population 2011'][i]
C:\Users\blaze\AppData\Local\Temp/ipykernel_11172/4071623889.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pop_merged2['LSOA population'][i] = df_pop_merged2['Population 2012'][i]
C:\Users\blaze\AppData\Local\Temp/ipykernel_11172/4071623889.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

In [54]:
df_pop_merged2.drop(['Population 2011','Population 2012', 'Population 2013','Population 2014','Population 2015','Population 2016','Population 2017', 'Population 2018', 'Population 2019'], axis=1, inplace=True)

In [55]:
df_pop_merged2 = df_pop_merged2.rename(columns={'Area Codes': 'LSOA code'})

# Population District (yearly)

In [56]:
df_population = pd.read_csv("Population of each district.csv")
df_population=df_population.rename(columns={'ladcode21':'Geo code', 'laname21':'name'})

df_districts = pd.read_csv("districts.csv")
df_districts = df_districts.rename(columns={'LAD19CD':'Geo code', 'LAD19NM':'name','PFA19CD': 'Geo code police', 'PFA19NM':'Force Name'})
df_districts.drop(['CSP19CD', 'CSP19NM', 'FID'], axis=1, inplace=True)

df_population2 = df_population.groupby(['name', 'Geo code'])[['population_2011','population_2012', 'population_2013', 'population_2014', 'population_2015', 'population_2016', 'population_2017', 'population_2018', 'population_2019']].sum()
df_population2.reset_index(inplace=True)
df_population_final = pd.merge(df_population2, df_districts, on='Geo code')
df_population_final = df_population_final.rename(columns={'name_x': 'name'})
df_population_final.drop(['name_y'], axis=1, inplace=True)

In [57]:
del df_population
del df_districts
del df_population2

# Merge population with the police

In [58]:
df_police_final2 = df_police_final.unstack(level=0)
df_police_final2.reset_index(inplace=True)

In [59]:
df_pop_pol = pd.merge(df_police_final2, df_population_final, on='Force Name')
df_pop_pol.drop(('Geo code police', ''), axis=1, inplace=True)

C:\Users\blaze\AppData\Local\Temp/ipykernel_11172/1797237973.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left,1 on the right)
  df_pop_pol = pd.merge(df_police_final2, df_population_final, on='Force Name')


In [60]:
del df_police_final
del df_police_final2

# Add year population and headcount column to merged dataframe

In [61]:
df_pop_pol1 = df_pop_pol.copy()
df_pop_pol1 = df_pop_pol1.loc[df_pop_pol1.index.repeat(8)]
df_pop_pol1['year']= 2012
df_pop_pol1['population']= 0
df_pop_pol1['headcount'] = 0
df_pop_pol1.reset_index(inplace=True)

In [62]:
del df_pop_pol

In [63]:
for i in range (1, df_pop_pol1.shape[0]):
    if df_pop_pol1['index'][i] == df_pop_pol1['index'][i-1]:
        df_pop_pol1['year'][i] = df_pop_pol1['year'][i-1]+1

C:\Users\blaze\AppData\Local\Temp/ipykernel_11172/2307358565.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pop_pol1['year'][i] = df_pop_pol1['year'][i-1]+1


In [64]:
for i in range (0, df_pop_pol1.shape[0]):
    if df_pop_pol1['year'][i]==2012:
        df_pop_pol1['population'][i] = df_pop_pol1['population_2011'][i]
        df_pop_pol1['headcount'][i] = df_pop_pol1[('Total (headcount)', 2011)][i]
    if df_pop_pol1['year'][i]==2013:
        df_pop_pol1['population'][i] = df_pop_pol1['population_2012'][i]
        df_pop_pol1['headcount'][i] = df_pop_pol1[('Total (headcount)', 2012)][i]
    if df_pop_pol1['year'][i]==2014:
        df_pop_pol1['population'][i] = df_pop_pol1['population_2013'][i]
        df_pop_pol1['headcount'][i] = df_pop_pol1[('Total (headcount)', 2013)][i]
    if df_pop_pol1['year'][i]==2015:
        df_pop_pol1['population'][i] = df_pop_pol1['population_2014'][i]
        df_pop_pol1['headcount'][i] = df_pop_pol1[('Total (headcount)', 2014)][i]
    if df_pop_pol1['year'][i]==2016:
        df_pop_pol1['population'][i] = df_pop_pol1['population_2015'][i]
        df_pop_pol1['headcount'][i] = df_pop_pol1[('Total (headcount)', 2015)][i]
    if df_pop_pol1['year'][i]==2017:
        df_pop_pol1['population'][i] = df_pop_pol1['population_2016'][i]
        df_pop_pol1['headcount'][i] = df_pop_pol1[('Total (headcount)', 2016)][i]
    if df_pop_pol1['year'][i]==2018:
        df_pop_pol1['population'][i] = df_pop_pol1['population_2017'][i]
        df_pop_pol1['headcount'][i] = df_pop_pol1[('Total (headcount)', 2017)][i]
    if df_pop_pol1['year'][i]==2019:
        df_pop_pol1['population'][i] = df_pop_pol1['population_2018'][i]
        df_pop_pol1['headcount'][i] = df_pop_pol1[('Total (headcount)', 2018)][i]

C:\Users\blaze\AppData\Local\Temp/ipykernel_11172/43154787.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pop_pol1['population'][i] = df_pop_pol1['population_2011'][i]
C:\Users\blaze\AppData\Local\Temp/ipykernel_11172/43154787.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pop_pol1['headcount'][i] = df_pop_pol1[('Total (headcount)', 2011)][i]
C:\Users\blaze\AppData\Local\Temp/ipykernel_11172/43154787.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

In [65]:
df_pop_pol1.drop(['population_2011','population_2012', 'population_2013','population_2014','population_2015','population_2016','population_2017', 'population_2018', 'population_2019'], axis=1, inplace=True)
df_pop_pol1.drop(['index',('Force Name', ''),('Total (headcount)', 2011),('Total (headcount)', 2012), ('Total (headcount)', 2013), ('Total (headcount)', 2014), ('Total (headcount)', 2015), ('Total (headcount)', 2016),('Total (headcount)', 2017), ('Total (headcount)', 2018),('Total (headcount)', 2019)], axis=1, inplace=True)


# Merge population per lsoa with crime dataset

In [66]:
df_pop_merged2 = df_pop_merged2.rename(columns={'LSOA code':'Geo code'})

In [67]:
data = pd.merge(df_pop_merged2, crime_per_district, on=['year', 'Geo code'])

In [68]:
del df_pop_merged2
del crime_per_district

In [69]:
data.rename(columns={'name': 'LSOA name', 'Geo code':'LSOA code',}, inplace=True)

In [70]:
data.reset_index(inplace=True)

# Link LSOA and districts

In [71]:
eng1 = pd.read_csv('England1 postcodes.csv')
eng2 = pd.read_csv('England2 postcodes.csv')
eng3 = pd.read_csv('England3 postcodes.csv')
wale = pd.read_csv('Wales postcodes.csv')
scot = pd.read_csv('Scotland postcodes.csv')
ire = pd.read_csv('BT postcodes.csv')
dist_lsoa = pd.concat([eng1, eng2, eng3, wale, scot, ire])

C:\Users\blaze\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (18,29,47) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\blaze\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (18,47) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\blaze\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [72]:
final_dist_lsoa = dist_lsoa[['District Code', 'LSOA Code']]

In [73]:
del eng1
del eng2
del eng3
del wale
del scot
del ire
del dist_lsoa

In [74]:
final_dist_lsoa.rename(columns={'LSOA Code':'LSOA code', 'District Code':'Geo code'}, inplace=True)

In [75]:
import gc
gc.collect()

118

# Merge linked districts + LSOA's and district population + police

In [76]:
data3 = pd.merge(df_pop_pol1, final_dist_lsoa, on=['Geo code'])

In [77]:
del final_dist_lsoa
del df_pop_pol1

In [78]:
data3.drop_duplicates(inplace=True)

In [79]:
data2 = pd.merge(data3, data, on=['year', 'LSOA code'])

In [80]:
del data3
del data

In [81]:
data2.drop(['Force Name_y', 'level_0','index'], axis=1, inplace=True)

In [82]:
data2.rename(columns={'Force Name_x':'Force Name'}, inplace=True)

# Read gdp data (yearly per district)

In [83]:
df_gdp = pd.read_excel("GDP_UK_per_head.xlsx")


In [84]:
df_gdp.rename(columns={'2019\n[note 3]':2019}, inplace=True)

In [85]:
ukdpg = df_gdp.copy()
ukdpg = ukdpg.loc[ukdpg.index.repeat(8)]
ukdpg['year']= 2012
ukdpg['GDP']= 0
ukdpg.reset_index(inplace=True)

In [86]:
del df_gdp

In [87]:
for i in range (1, ukdpg.shape[0]):
    if ukdpg['index'][i] == ukdpg['index'][i-1]:
        ukdpg['year'][i] = ukdpg['year'][i-1]+1

C:\Users\blaze\AppData\Local\Temp/ipykernel_11172/1383387713.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ukdpg['year'][i] = ukdpg['year'][i-1]+1


In [88]:
for i in range (0, ukdpg.shape[0]):
    if ukdpg['year'][i]==2012:
        ukdpg['GDP'][i] = ukdpg[2011][i]
    if ukdpg['year'][i]==2013:
        ukdpg['GDP'][i] = ukdpg[2012][i]
    if ukdpg['year'][i]==2014:
        ukdpg['GDP'][i] = ukdpg[2013][i]
    if ukdpg['year'][i]==2015:
        ukdpg['GDP'][i] = ukdpg[2014][i]
    if ukdpg['year'][i]==2016:
        ukdpg['GDP'][i] = ukdpg[2015][i]
    if ukdpg['year'][i]==2017:
        ukdpg['GDP'][i] = ukdpg[2016][i]
    if ukdpg['year'][i]==2018:
        ukdpg['GDP'][i] = ukdpg[2017][i]
    if ukdpg['year'][i]==2019:
        ukdpg['GDP'][i] = ukdpg[2018][i]

C:\Users\blaze\AppData\Local\Temp/ipykernel_11172/1553439139.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ukdpg['GDP'][i] = ukdpg[2011][i]
C:\Users\blaze\AppData\Local\Temp/ipykernel_11172/1553439139.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ukdpg['GDP'][i] = ukdpg[2012][i]
C:\Users\blaze\AppData\Local\Temp/ipykernel_11172/1553439139.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ukdpg['GDP'][i] = ukdpg[2013][i]
C

In [89]:
ukdpg.drop(['index', 'ITL1 Region', 'LA name', 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019], axis=1, inplace=True)
ukdpg = ukdpg.rename(columns={'LA code': 'Geo code'})

# Merge gdp with main dataframe

In [90]:
data_final2 = pd.merge(data2, ukdpg,  on=['Geo code', 'year'])

In [91]:
del data2
del ukdpg

# READ IMD data (same every year (measured every 7 years, we have data from 2015), per LSOA) and merge it with previously merged data

In [92]:
df_IMD = pd.read_csv("IMD_2015_LSOA_score.csv", skiprows = 7)

df_IMD = df_IMD[["Reference area", "2015"]]
df_IMD = df_IMD.rename(columns={"Reference area":"LSOA name", "2015" : "IMD score"})

In [93]:
data_final = pd.merge(data_final2, df_IMD,  on=['LSOA name']).dropna()
data_final.reset_index(inplace=True)

In [94]:
del data_final2
del df_IMD

In [95]:
data_final.drop(['index'], axis=1, inplace=True)

# Rename and save everything as pickle (python 3.8 or higher to read it)

In [96]:
d = {'population' : 'population_last_year', 'headcount': 'headcount_last_year', 
     'LSOA population': 'LSOA population_last_year', 'GDP': 'GDP_last_year'}

data_final.rename(columns=d, inplace=True)

In [97]:
data_final

,Force Name,name,Geo code,Geo code police,year,population_last_year,headcount_last_year,LSOA code,LSOA population_last_year,Month,LSOA name,amount_of_crimes,month,prev_month_crimes,prev_year_crimes,GDP_last_year,IMD score
0,Bedfordshire,Bedford,E06000055,E23000026,2012,157840,1238,E01017466,1937.0,2012-01,Bedford 012A,132,1,133.0,110.0,22533,49.525
1,Bedfordshire,Bedford,E06000055,E23000026,2012,157840,1238,E01017466,1937.0,2012-02,Bedford 012A,111,2,132.0,129.0,22533,49.525
2,Bedfordshire,Bedford,E06000055,E23000026,2012,157840,1238,E01017466,1937.0,2012-03,Bedford 012A,147,3,111.0,159.0,22533,49.525
3,Bedfordshire,Bedford,E06000055,E23000026,2012,157840,1238,E01017466,1937.0,2012-04,Bedford 012A,117,4,147.0,181.0,22533,49.525
4,Bedfordshire,Bedford,E06000055,E23000026,2012,157840,1238,E01017466,1937.0,2012-05,Bedford 012A,134,5,117.0,147.0,22533,49.525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2764045,Wiltshire,Wiltshire,E06000054,E23000038,2019,498064,1021,E01032697,1624.0,2019-08,Wiltshire 041G,13,8,14.0,19.0,26113,16.378
2764046,Wiltshire,Wiltshire,E06000054,E23000038,2019,498064,1021,E01032697,1624.0,2019-09,Wiltshire 041G,11,9,13.0,17.0,26113,16.378
2764047,Wiltshire,Wiltshire,E06000054,E23000038,2019,498064,1021,E01032697,1624.0,2019-10,Wiltshire 041G,18,10,11.0,29.0,26113,16.378
2764048,Wiltshire,Wiltshire,E06000054,E23000038,2019,498064,1021,E01032697,1624.0,2019-11,Wiltshire 041G,14,11,18.0,22.0,26113,16.378


In [99]:
data_final.to_pickle("crime_data_final.pkl") 

# Check how much data did we lost: 15.86% of data, I have no idea where and how

In [15]:

(1 - (data_final.shape[0]/crime_per_district.shape[0]))*100

15.864645587752745

In [6]:
conn.close()